# Periodogram viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***
* *First block is to link google drive to colab so that it can find the data*
* *General note, all blocks run much quicker locally, with some more convenience functions included...*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#this is so that the util .py files can be found
#it still doesn't let you read in files directly from here though... need the full paths
import sys
sys.path.append('/content/drive/My Drive/HOYS_Colab/HOYS_period')

* *if any packages dont load, add a !pip install command with the package*

In [ ]:
#load required modules
#online viewer binder requires the install commands for first run, ignore erros if running locally
!pip install pandas
!pip install astropy
!pip install plotly
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

*List available lightcurve data and select from dropdown list*

In [ ]:
#read HOYS csv file
lc_folder = "/content/drive/My Drive/HOYS_Colab/HOYS_period/light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
)
display(lc_select)



Dropdown(description='Select a light curve file:', options=('lightcurve_304.493453_37.933344.txt', 'lightcurve…

In [ ]:
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
#view file head and tail
lc_data

,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id
0,157901959,15.9468,0.072742,0.0258,2109.906006,1625.750732,312.755689,44.260768,0.003400,0.0,15.7621,23915,V,Visual,2.458877e+06,7,2,[118] IC5070_incl_201_V2492Cyg,25838
1,68911857,14.3734,0.114608,0.0107,726.341125,962.783325,312.756257,44.260777,0.004735,0.0,13.3676,6571,I,-SI-,2.458370e+06,33,56,[118] IC5070_incl_201_V2492Cyg,6460
2,9712719,16.3896,0.090909,0.0133,1925.137207,2513.876953,312.757468,44.260779,0.000891,3.0,15.6094,771,R,Red,2.457615e+06,7,2,[118] IC5070_incl_201_V2492Cyg,717
3,69518711,14.0960,0.058415,0.0315,274.660004,380.768188,312.756940,44.260781,0.003053,0.0,15.0650,6807,I,I BAND,2.458373e+06,35,41,[118] IC5070_incl_201_V2492Cyg,6675
4,82546481,14.5852,0.053607,0.0093,2103.738525,1601.775269,312.756380,44.260781,0.000834,3.0,15.6960,8360,I,I-Band,2.458437e+06,7,2,[118] IC5070_incl_201_V2492Cyg,8184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,73171336,18.1260,0.154783,0.1881,997.796570,684.403320,312.756958,44.262391,0.001195,0.0,17.6873,7338,B,B,2.458366e+06,41,51,[118] IC5070_incl_201_V2492Cyg,7273
4686,66751098,-99.3000,0.000000,-99.0000,505.185791,2.508300,312.756545,44.262412,0.004569,283.0,-99.0000,5763,V,V,2.458347e+06,10,8,[118] IC5070_incl_201_V2492Cyg,5541
4687,145854888,16.2533,0.197534,0.0353,2637.276611,2156.505371,312.756633,44.262426,0.001426,2.0,14.9249,16116,V,Green,2.458776e+06,54,127,[118] IC5070_incl_201_V2492Cyg,18481
4688,60650715,14.7058,0.034175,0.0046,2019.585815,2487.909668,312.756043,44.262428,0.000837,3.0,15.1609,4648,I,I-Band,2.458316e+06,7,2,[118] IC5070_incl_201_V2492Cyg,4486


* *select filter and plot lightcurve*
* *could have widgets here for selecting filter*
* *slider for calibrated error, 0 - 0.3*
* slider to remove magnitude extremes/outliers
* time value slider from 10 days to 1 year, default to 50 days
* jd slider from to default to max



In [ ]:
#select data from specifed filter and remove bad data
i_data=lc_data[(lc_data['filter']=='V') & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < 0.1) & (lc_data['date'] > 2400000) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]

#median filter the lightcurve over a time window
window = 50.0 #half the time window in days
mags = numpy.array(i_data['calibrated_magnitude'])
times = numpy.array(i_data['date'])
med = numpy.zeros(len(i_data))
for i in range(0,len(med)):
  check = numpy.where( numpy.abs(times[i] - times) < window )
  if (len(check[0]) > 0):
    med[i] = numpy.median(mags[check[0]])
i_data['calibrated_magnitude'] = i_data['calibrated_magnitude'] - med + numpy.median(i_data['calibrated_magnitude'])

#plot lightcurve
fig1=px.scatter(i_data,x=i_data.date,y=i_data.calibrated_magnitude)
fig1['layout']['yaxis']['autorange'] = "reversed"
iplot(fig1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



* *plot the periodogram*
* check using .power with a freq array instead of .autopower()

In [ ]:
#generate and plot periodogram
#jd=Time(i_data.date,format='jd') #was using the astropy time units for the date
jd=i_data.date
ls=LombScargle(jd,i_data.calibrated_magnitude)
frequency,power=ls.autopower()
fig2=px.line(x=1/frequency,y=power,labels={'x':'Period [days]','y':'Power'})
fig2.update_xaxes(range=[0, (max(jd)-min(jd))/4.]) #added max range for plot, change if a better range is more useful
iplot(fig2)
#best_period = 1/frequency[numpy.argmax(power[numpy.where(1./frequency < 1000.0)])]
best_period = 1/frequency[numpy.argmax(power)]
print('best period:',numpy.round(best_period,5),'days')

best period: 7.36518 days


* *plot the phase plot and sine-fit*
* change period slider to include more digits


In [ ]:
time = i_data['date']
phase = numpy.zeros(len(time))

#determine start values for the paramers from the data
offset0 = numpy.median(i_data['calibrated_magnitude'])
amp0 = numpy.std(i_data['calibrated_magnitude']) * 1.414
period0 = best_period
#trying to add interactive sliders for the sine-function - needs period, amp, offset and phase0 to be variable
def view_image(period=period0, amp=amp0, offset=offset0, phase0=0.5):
  
  phase = numpy.mod( time/period , 1.0 )
  #add a fitted sine-function
  amp = amp
  phase0 = phase0
  offset= offset
  fit = offset + amp * numpy.sin(2.0*math.pi*(-phase0 + phase ))

  #create dataframe with the phase, magnitude and fit values
  #use append to expand the size of it all to include phase+1.0
  df_phase=pd.DataFrame({'phase':phase.append(phase+1.0), 'i_data':i_data['calibrated_magnitude'].append(i_data['calibrated_magnitude']), 'fit':fit.append(fit)})
  #melt the dataframe by phase as the id variable so that the data can be plot and coloured by whether it's the data or the fit
  df_melt = df_phase.melt(id_vars='phase', value_vars=['i_data', 'fit'],value_name='calibrated_magnitude')

  fig4=px.scatter(df_melt,x='phase', y='calibrated_magnitude',color='variable')
  fig4['layout']['yaxis']['autorange'] = "reversed"
  
  fig4.update_layout(
    autosize=False,
    width=1000,
    height=600,)

  iplot(fig4)

interact(view_image, period=(period0-0.01, period0+0.01, 0.0001), amp=(amp0-0.3, amp0+0.3, 0.001), offset=(offset0-0.2,offset0+0.2,0.001), phase0=(0,1,0.01))


interactive(children=(FloatSlider(value=7.365183488093845, description='period', max=7.375183488093845, min=7.…

<function __main__.view_image>